# Banking Customer Churn Prediction

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import plotly.express as px

## 2. Data Collection and Exploration

In [2]:
# load data
df = pd.read_csv(r"D:\AI\data\datasets-1\Bank_churn_modelling.csv")
df.shape

(10000, 14)

In [3]:
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0


In [4]:
df.describe()

,RowNumber,CustomerId,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
count,10000.00000,1.000000e+04,10000.000000,10000.000000,9985.000000,10000.000000,10000.000000,10000.00000,10000.000000,9990.000000,10000.000000
mean,5000.50000,1.569094e+07,650.528800,38.921800,5.013721,76485.889288,1.530200,0.70550,0.515100,100127.441333,0.203700
std,2886.89568,7.193619e+04,96.653299,10.487806,2.892213,62397.405202,0.581654,0.45584,0.499797,57497.228065,0.402769
min,1.00000,1.556570e+07,350.000000,18.000000,0.000000,0.000000,1.000000,0.00000,0.000000,11.580000,0.000000
25%,2500.75000,1.562853e+07,584.000000,32.000000,3.000000,0.000000,1.000000,0.00000,0.000000,51112.885000,0.000000
50%,5000.50000,1.569074e+07,652.000000,37.000000,5.000000,97198.540000,1.000000,1.00000,1.000000,100238.110000,0.000000
75%,7500.25000,1.575323e+07,718.000000,44.000000,7.000000,127644.240000,2.000000,1.00000,1.000000,149400.922500,0.000000
max,10000.00000,1.581569e+07,850.000000,92.000000,10.000000,250898.090000,4.000000,1.00000,1.000000,199992.480000,1.000000


Analysis:
    
        - RowNumber, customerid and surname are unwanted columns
        - Label/target: Exited
        - demographic: Age, Gender, Geography
        - Financial capability: CreditScore, EstimatedSalary, Balance
        - Relation with Bank: Tenure, NumofProducts, IsActiveMember, HasCrCard
        - Age may have some outliers

## 3. Data Cleaning

### Handling missing data
    - data is missing because it does not exist
        - Numeric: convert the column into binary or categorical, add one category as "unavailable"/"others"
        - Categoric: replace all missing values by an additional/existing category "others"/"not known"
        
        
    - data exists, but it is missing because of some human or system error
        - if any row has more than 60% of values missing - drop the row
        - if any column has more than 70%-80% values missing - drop the column
        - for rows having upto 5%-10% (depending on size of data) missing data - statistical imputation
            - Categoric: Nominal: Mode
            - Categoric: Ordinal: Median
            - Numeric: skewness > 0.1 or skewness < -0.1 => median
            - NumericL: skewness is b/w -0.1 to +0.1 => Mean
          - for rows having more than 10% of data missing: ML based imputation
          

In [5]:
# check for missing values
df.isnull().sum()

RowNumber           0
CustomerId          0
Surname             0
CreditScore         0
Geography           0
Gender              0
Age                 0
Tenure             15
Balance             0
NumOfProducts       0
HasCrCard           0
IsActiveMember      0
EstimatedSalary    10
Exited              0
dtype: int64

In [6]:
df[['Tenure','EstimatedSalary']].skew()

Tenure             0.010333
EstimatedSalary    0.001322
dtype: float64

In [7]:
df.Tenure.fillna(df.Tenure.mean(),inplace=True)
df.EstimatedSalary.fillna(df.EstimatedSalary.mean(),inplace=True)

In [8]:
df.isnull().sum()

RowNumber          0
CustomerId         0
Surname            0
CreditScore        0
Geography          0
Gender             0
Age                0
Tenure             0
Balance            0
NumOfProducts      0
HasCrCard          0
IsActiveMember     0
EstimatedSalary    0
Exited             0
dtype: int64

In [9]:
# check for duplicates
df.duplicated().sum()

0

In [10]:
# duplicate check based on specific column
df['CustomerId'].duplicated().sum()

0

In [11]:
print(df.shape)
# drop unwanted column
df.drop(columns=['RowNumber','Surname','CustomerId'],inplace=True)
print(df.shape)

(10000, 14)
(10000, 11)


In [12]:
# check for outliers
df.skew()

C:\Users\admin\AppData\Local\Temp/ipykernel_4156/3256115495.py:2: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df.skew()


CreditScore       -0.071607
Age                1.011320
Tenure             0.010341
Balance           -0.141109
NumOfProducts      0.745568
HasCrCard         -0.901812
IsActiveMember    -0.060437
EstimatedSalary    0.001323
Exited             1.471611
dtype: float64

## 4. Feature Selection